In [32]:
import pandas as pd
import numpy as np
from sklearn import cross_validation, preprocessing, metrics
df_train = pd.read_csv('../Data_with_NumMosquitos/unique_train.csv')
#df_train_nomosq = pd.read_csv('../train_filled_new.csv')
df_test = pd.read_csv('../Data_with_NumMosquitos/new_test_priors.csv')
#df_test_nomosq = pd.read_csv('../test_filled_new.csv')

In [33]:
random_state = 42
df_train = df_train.drop(['Block'],axis=1)
df_test = df_test.drop(['Block'],axis=1)

In [34]:
def year_train_test_split(train, target, year):

    # Create copy
    X = train.copy()

    # Retrieve target column and remove from X
    y = X[target]
    X.drop([target], axis=1)

    # Create mask
    msk = X.Year == year

    # Drop date column
    X = X.drop(['Year', 'WnvPresent_DateTrapSpecies'], axis=1)
    
    # Create dfs based on mask    
    X_train = X[~msk]
    X_test = X[msk]
    X_test = X_test.drop(['count_mosquitos'], axis=1)
    y_train = y[~msk]
    y_test = y[msk]
    y_train_numMosquitos = X.count_mosquitos[~msk] 
    y_test_numMosquitos = X.count_mosquitos[msk] 
    
    return X_train, X_test, y_train, y_test, y_train_numMosquitos, y_test_numMosquitos,msk

In [35]:
list1 = [#'NumMosquitos',
                            'Trap',
                            'Month',
                            'Block',
                            'AddressAccuracy',
                            'AddressNumberAndStreet',
                            'Address',
                            'Street',
                            'CodeSum'
]

[var for var in list1 if var in df_train.columns]

['Month', 'AddressAccuracy']

In [36]:
train_for_loo = df_train.drop(['Month', 'AddressAccuracy'], axis=1)
test_for_loo = df_test.drop(['Month','AddressAccuracy','Unnamed: 0','Year', 'count_mosquitos'],axis=1)
[var for var in test_for_loo.columns if var not in train_for_loo], 
[var for var in train_for_loo.columns if var not in test_for_loo]

['Year', 'count_mosquitos', 'WnvPresent_DateTrapSpecies']

In [37]:
y_clf = train_for_loo.WnvPresent_DateTrapSpecies
yhat_clf = pd.DataFrame(df_test.Tavg)
#yhat_clf.Tavg

In [52]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
#XgBoost
#NN

#clf = KNeighborsClassifier(n_neighbors=10)
clf = RandomForestClassifier(n_estimators = 400,n_jobs = -1)

In [56]:
#y_clf = train_for_loo.WnvPresent_DateTrapSpecies
#y_clf = 'NaN'

for year in [2007, 2009, 2011, 2013]:

    X_train,X_test, y_train, y_test, y_train_numMosquitos, y_test_numMosquitos,year_mask = year_train_test_split(
            train_for_loo,
            'WnvPresent_DateTrapSpecies',
            year) 
    X_train = X_train.drop('count_mosquitos',axis=1)
    
    clf.fit(X_train,y_train)
    y_clf[year_mask] =clf.predict_proba(X_test)[:,1]
    #test3 = clf.predict_proba(X_train)
    #test2 = clf.predict_proba(X_test)
    yhat_clf[year]= clf.predict_proba(test_for_loo)[:,1]

#    test = clf.predict_proba(test_for_loo)
try:
    yhat_clf = yhat_clf.drop('Tavg',axis=1)
except: 
    print('already droped')

already droped


In [69]:
yhat_clf.mean(axis=1)

0         0.071250
1         0.106250
2         0.077500
3         0.108125
4         0.088125
5         0.108125
6         0.071250
7         0.090000
8         0.072500
9         0.105625
10        0.075000
11        0.107500
12        0.088125
13        0.107500
14        0.072500
15        0.088750
16        0.068750
17        0.110625
18        0.068750
19        0.110625
20        0.093750
21        0.110625
22        0.068750
23        0.086250
24        0.068750
25        0.110625
26        0.068125
27        0.110625
28        0.091875
29        0.110625
            ...   
116263    0.080625
116264    0.125000
116265    0.103750
116266    0.125000
116267    0.083750
116268    0.108125
116269    0.075000
116270    0.108125
116271    0.080000
116272    0.108125
116273    0.092500
116274    0.108125
116275    0.075000
116276    0.094375
116277    0.070625
116278    0.096250
116279    0.077500
116280    0.096250
116281    0.086250
116282    0.096250
116283    0.070625
116284    0.

In [73]:
y_clf

0       0.0550
1       0.0325
2       0.0275
3       0.0400
4       0.0400
5       0.0425
6       0.0175
7       0.0375
8       0.0375
9       0.0375
10      0.0375
11      0.0375
12      0.0375
13      0.0375
14      0.0375
15      0.0425
16      0.0475
17      0.0375
18      0.0325
19      0.0400
20      0.0400
21      0.0400
22      0.0375
23      0.0425
24      0.0400
25      0.1675
26      0.0700
27      0.1625
28      0.1500
29      0.0550
         ...  
8491    0.4750
8492    0.4700
8493    0.4700
8494    0.4700
8495    0.4700
8496    0.4700
8497    0.4725
8498    0.4700
8499    0.4725
8500    0.4725
8501    0.4725
8502    0.4725
8503    0.4750
8504    0.4700
8505    0.4725
8506    0.4700
8507    0.4725
8508    0.4725
8509    0.4725
8510    0.4775
8511    0.4900
8512    0.4700
8513    0.4700
8514    0.4750
8515    0.4900
8516    0.4700
8517    0.4700
8518    0.4900
8519    0.4775
8520    0.4875
Name: WnvPresent_DateTrapSpecies, dtype: float64

In [76]:
df_train['RF_Prediction'] = y_clf
df_test['RF_Prediction'] = yhat_clf.mean(axis=1)

In [77]:
df_test['RF_Prediction']

0         0.071250
1         0.106250
2         0.077500
3         0.108125
4         0.088125
5         0.108125
6         0.071250
7         0.090000
8         0.072500
9         0.105625
10        0.075000
11        0.107500
12        0.088125
13        0.107500
14        0.072500
15        0.088750
16        0.068750
17        0.110625
18        0.068750
19        0.110625
20        0.093750
21        0.110625
22        0.068750
23        0.086250
24        0.068750
25        0.110625
26        0.068125
27        0.110625
28        0.091875
29        0.110625
            ...   
116263    0.080625
116264    0.125000
116265    0.103750
116266    0.125000
116267    0.083750
116268    0.108125
116269    0.075000
116270    0.108125
116271    0.080000
116272    0.108125
116273    0.092500
116274    0.108125
116275    0.075000
116276    0.094375
116277    0.070625
116278    0.096250
116279    0.077500
116280    0.096250
116281    0.086250
116282    0.096250
116283    0.070625
116284    0.

In [78]:
df_train.to_csv('train_plus_pred.csv',index=False)
df_test.to_csv('test_plus_pred.csv',index=False)